In [8]:
!pip3 install groq
!pip3 install PyPDF2 langchain==0.3.9 langchain-community==0.3.9 langchain-core==0.3.21 langchain-experimental==0.3.3 huggingface-hub==0.23.2 instructor==1.7.0 InstructorEmbedding==1.0.1 sentence-transformers==2.2.2 torch pdfplumber faiss-cpu
!pip3 install langchain_groq

  Using cached langchain_core-0.3.21-py3-none-any.whl.metadata (6.3 kB)
Using cached langchain_core-0.3.21-py3-none-any.whl (409 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.30
    Uninstalling langchain-core-0.3.30:
      Successfully uninstalled langchain-core-0.3.30


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-groq 0.2.3 requires langchain-core<0.4.0,>=0.3.29, but you have langchain-core 0.3.21 which is incompatible.


  Using cached langchain_core-0.3.30-py3-none-any.whl.metadata (6.3 kB)
Using cached langchain_core-0.3.30-py3-none-any.whl (411 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.21
    Uninstalling langchain-core-0.3.21:
      Successfully uninstalled langchain-core-0.3.21


In [19]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_experimental.text_splitter import SemanticChunker
import os
import glob
from langchain.embeddings import OllamaEmbeddings
import pdfplumber
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from InstructorEmbedding import INSTRUCTOR
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ollama import ChatOllama
from langchain_ollama.llms import OllamaLLM
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings import SentenceTransformerEmbeddings

In [20]:
from dotenv import load_dotenv
load_dotenv()
api_key= os.getenv('GROQ_API_KEY')

In [21]:
model_name = 'llama-3.1-70b-versatile'
#model_name = 'llama3-8b-8192'

groq_chat = ChatGroq(
        groq_api_key=api_key,
        model_name=model_name
    )

llm = groq_chat

In [36]:
## Do not mofify
def load_db(embeddings, files_path):
    files = glob.glob(f'{files_path}*.pdf')
    text =''
    for file in files:
        with open(file,'rb') as f:
            pdf_reader = PdfReader(f)
            for page in pdf_reader.pages:
                text += page.extract_text()

    text_splitter=SemanticChunker(
        embeddings, breakpoint_threshold_type="percentile")
    #text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_text(text)
    # define embedding
    vectorstore = FAISS.from_texts(docs, embeddings)
    return vectorstore

In [37]:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [38]:
files_path = './labs/'

In [39]:
#Do not modify 
if not os.path.exists('faiss_index_labs'):
    vectorstore=load_db(embeddings,files_path)
    vectorstore.save_local("faiss_index_labs")
else:
    vectorstore = FAISS.load_local("faiss_index_labs",embeddings=embeddings,allow_dangerous_deserialization=True)

retriever = vectorstore.as_retriever()

In [45]:
template = """
 Rol: Asistente de Laboratorio

 Indicaciones: Eres un asistente que puede responder preguntas acerca de las guías de laboratorio
 del curso F1008, el usuario te preguntará sobre la fecha, los objetivos, el contenido, 
 así como del procedimiento. 
 Aunque las guías están en inglés deberás contestar siempre en español. 
 Si las preguntas son sobre otro tema, contesta que no puedes contestar a esa pregunta.

Si la pregunta es descortés puedes indicarle al usuario que esa no es la forma de preguntar, dile que debe
reescribir su pregunta.

{context}

Pregunta: {question}
"""
qa_prompt = ChatPromptTemplate.from_template(template)

In [46]:
#Do not modify
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": qa_prompt}
)

In [47]:
history = []

In [48]:
def handle_conversation(conversation,history):
    print('Escribe "exit" para salir')
    while True:
        question = input('Tu: ')
        if question.lower() == 'exit':
            break
        print(question)
        result = conversation({"question": question}, {"chat_history": history})
        history.append((question, result["answer"]))
        print('---------------------------------------------------------\n\n',"Bot:\n ",result['answer'],'\n---------------------------------------------------------\n\n')

handle_conversation(conversation_chain,history)

Escribe "exit" para salir
¿Acerca de que tema es la guía de Modos Normales?
---------------------------------------------------------

 Bot:  La guía de laboratorio se enfoca en el tema de "Modos Normales en Cuerdas Vibrantes" (Normal Modes in Vibrating Strings). Este experimento busca estudiar y comprender los modos normales en cuerdas vibrantes, utilizar herramientas estadísticas para estimar la velocidad de propagación de las ondas y analizar los patrones de vibración en cuerdas sometidas a diferentes tensiones y frecuencias. 
---------------------------------------------------------


Por favor indicame el procedimiento de la p´ractica de laboratorio para hacerla lo más eficeintemente posible.
---------------------------------------------------------

 Bot:  El procedimiento óptimo para realizar la práctica de laboratorio de Modos Normales en Cuerdas Vibrantes de manera eficiente es el siguiente:

1. **Preparación**: Antes de comenzar la práctica, es importante leer y comprender la